In [ ]:
!pip install requests beautifulsoup4

In [1]:
from bs4 import BeautifulSoup
import requests

In [2]:
url = 'https://forum.egosoft.com/search.php?search_author=j.harshaw'

In [3]:
response = requests.get(url)

In [4]:
content = BeautifulSoup(response.content, 'html5lib')

In [5]:
posts = []
searchPages = []

for a in content.find_all('a'):
    if a.has_attr('href'):
        href = a['href']
        if 'p=' in href:
            posts.append(href)
        if 'search_id=' in href:
            searchPages.append(href)

In [6]:
posts

['viewtopic.php?p=4719880&highlight=#4719880',
 'viewtopic.php?p=4719838&highlight=#4719838',
 'viewtopic.php?p=4715880&highlight=#4715880',
 'viewtopic.php?p=4715874&highlight=#4715874',
 'viewtopic.php?p=4715864&highlight=#4715864',
 'viewtopic.php?p=4715724&highlight=#4715724',
 'viewtopic.php?p=4715331&highlight=#4715331',
 'viewtopic.php?p=4715259&highlight=#4715259',
 'viewtopic.php?p=4713789&highlight=#4713789',
 'viewtopic.php?p=4712921&highlight=#4712921',
 'viewtopic.php?p=4712915&highlight=#4712915',
 'viewtopic.php?p=4712897&highlight=#4712897',
 'viewtopic.php?p=4711346&highlight=#4711346',
 'viewtopic.php?p=4706236&highlight=#4706236',
 'viewtopic.php?p=4706177&highlight=#4706177']

In [7]:
searchPages

['search.php?search_id=517578610&start=15',
 'search.php?search_id=517578610&start=30',
 'search.php?search_id=517578610&start=45',
 'search.php?search_id=517578610&start=60',
 'search.php?search_id=517578610&start=75',
 'search.php?search_id=517578610&start=90',
 'search.php?search_id=517578610&start=105',
 'search.php?search_id=517578610&start=120',
 'search.php?search_id=517578610&start=15']

In [19]:
post = posts[1]

In [20]:
postId = post.split('#')[1]

In [21]:
postId

'4719838'

In [22]:
postResponse = requests.get('https://forum.egosoft.com/' + post)

In [23]:
postContent = BeautifulSoup(postResponse.content, 'html5lib')

In [24]:
post

'viewtopic.php?p=4719838&highlight=#4719838'

In [25]:
anchorPost = postContent.find('a', {'name': postId})

In [26]:
anchorPost

<a name="4719838"></a>

In [27]:
postRow = anchorPost.parent.parent.parent

In [28]:
postbody = postRow.find_all('span', {'class': 'postbody'})
postdetails = postRow.find_all('span', {'class': 'postdetails'})

In [43]:
postdetail = postdetails[1]

Posted: Wed, 25. Jul 18, 16:33    Post subject: Re: Bunch of technical questions related to weapons/shields


In [44]:
import re

In [53]:
x = re.findall('Posted: (.+)Post subject: (.+)', postdetail.text)[0]
print(x[0].rstrip()) # publication date
print(x[1].rstrip()) # topic title

Wed, 25. Jul 18, 16:33
Re: Bunch of technical questions related to weapons/shields


In [37]:
for pb in postbody:
    print(pb.text.rstrip())
    print('-----------------------')


-----------------------


Yes. Number of guns, engines, shield generators, turrets that can be installed is defined on the hull.
-----------------------


Yes, but not sure if you can assign firing of each group to different keys.  There are certainly two groups that are assigned to different keys.  But i think you have to switch between groups before being able to use a key to fire weapons in that group.  Any given weapon can be in several groups.  Not all groups have to be populated.
-----------------------


At the moment, ship hulls have missile storage which are internal bays only for missiles.
-----------------------


All ships that fire missiles currently use ammunition and cannot do so when they run out.
-----------------------


Not sure i understand your question?  Each ship can have a certain number of shield generators (variable, but max depends on the hull), each shield generator is rated to a certain amount of joules, total effect is cumulative EXCEPT for smaller shield

In [54]:
postbody

[<span class="postbody"></span>, <span class="postbody">
 <br/>
 Yes. Number of guns, engines, shield generators, turrets that can be installed is defined on the hull.
 <br/>
 
 <br/>
 </span>, <span class="postbody">
 <br/>
 Yes, but not sure if you can assign firing of each group to different keys.  There are certainly two groups that are assigned to different keys.  But i think you have to switch between groups before being able to use a key to fire weapons in that group.  Any given weapon can be in several groups.  Not all groups have to be populated.
 <br/>
 
 <br/>
 </span>, <span class="postbody">
 <br/>
 At the moment, ship hulls have missile storage which are internal bays only for missiles.
 <br/>
 
 <br/>
 </span>, <span class="postbody">
 <br/>
 All ships that fire missiles currently use ammunition and cannot do so when they run out.
 <br/>
 
 <br/>
 </span>, <span class="postbody">
 <br/>
 Not sure i understand your question?  Each ship can have a certain number of shield 

In [56]:
for pb in postbody:
    for c in pb.children:
        print(c)



<br/>

Yes. Number of guns, engines, shield generators, turrets that can be installed is defined on the hull.

<br/>



<br/>




<br/>

Yes, but not sure if you can assign firing of each group to different keys.  There are certainly two groups that are assigned to different keys.  But i think you have to switch between groups before being able to use a key to fire weapons in that group.  Any given weapon can be in several groups.  Not all groups have to be populated.

<br/>



<br/>




<br/>

At the moment, ship hulls have missile storage which are internal bays only for missiles.

<br/>



<br/>




<br/>

All ships that fire missiles currently use ammunition and cannot do so when they run out.

<br/>



<br/>




<br/>

Not sure i understand your question?  Each ship can have a certain number of shield generators (variable, but max depends on the hull), each shield generator is rated to a certain amount of joules, total effect is cumulative EXCEPT for smaller shield generators th